In [1]:
#2.0: import pdfplumber, regex, etc. (mesmo que antes, sem alteração)
!pip install -qq pdfplumber openpyxl regex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 47.0 MB/s eta 0:00:00


In [2]:
import pdfplumber
import regex
import pandas as pd
from google.colab import files

In [3]:
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]  # nome do arquivo PDF
print("PDF carregado:", pdf_path)

extracted_data = []

Saving concarga2.pdf to concarga2.pdf
PDF carregado: concarga2.pdf


In [4]:
#2.3: Atualização da expressão regular para servidor:
# Permite asteriscos no nome e opcionalmente a presença de "RegC" entre o CH e o Código SI.
servidor_regex = (
    r"^([\p{L}\*'\s\.]+)"    # Nome do servidor (inclui *), #2.3
    r"\s+(MAT|VES)"          # Turno, #2.3
    r"\s+(\d{2})"            # CH do servidor, #2.3
    r"(?:\s+RegC)?"          # opcional "RegC", #2.3
    r"\s+(\d{11,})$"         # Código SI (11+ dígitos), #2.3
)

mvh_regex = (
    r"Matric\.\:\s+([\d\-A-Z]+)\s+"   # Matrícula
    r"VINCULO\:\s+([A-Z])\s+"        # Vínculo
    r"Hab\.\:\s+(.+)$"               # Habilitação (resto da linha)
)

disc_pattern = (
    r"(0\d{2}\s[\p{L}]+)\s+(\d{2})\s+(\d{2})\s+(\d{4,})\s+([\p{L}]+)\s+(\d+)"
)

In [5]:
# =============================================================
# 5) LER O PDF COMO UM TODO E CRIAR UMA ÚNICA LISTA DE LINHAS
# =============================================================
#2.0: Substituímos a lógica de "for page in pdf.pages" por uma leitura global.
all_lines = []  #2.0
with pdfplumber.open(pdf_path) as pdf:  #2.0
    for page in pdf.pages:  #2.0
        text = page.extract_text()  #2.0
        if text:  #2.0
            page_lines = text.split("\n")  #2.0
            all_lines.extend(page_lines)  #2.0

#2.6: Ajuste na função de filtragem de cabeçalhos/rodapés
def is_header_or_footer(line):  #2.6
    # Mantém a lógica anterior:
    if regex.search(r"Quadro de Carga Horária", line):
        return True
    if "PAG." in line:
        return True
    if regex.search(r"^\d{2}:\d{2}:\d{2}", line):
        return True
    if regex.search(r"^Ano:\s*\d{4}", line):
        return True

    #2.6: Ignorar qualquer linha que contenha "Os2PDF"
    if "Os2PDF" in line:
        return True

    #2.6: Se for exatamente esta sequência de “fragmentos”
    # ou linhas muito curtas que não servem de dado real:
    # (caso sejam várias linhas separadas, cada uma “F”, “0”, “2”, etc.,
    # já estava sendo tratada por in {"F","0","2","B","V","5"}; mas se
    # porventura aparecerem juntas, podemos tratar também.)
    if line.strip() in {"F", "0", "2", "B", "V", "5", "F 0 2 B V 2 2 5"}:
        return True

    return False

#2.6: agora aplicamos a filtragem revisada
filtered_lines = []
for line in all_lines:
    if not is_header_or_footer(line):
        filtered_lines.append(line)

#2.6: Preprocessamento para juntar linhas quebradas – logicamente igual a 2.5,
# mas tendo removido mais “lixo” na filtragem anterior, a merge tende a ficar mais correta.
merged_lines = []
for line in filtered_lines:
    stripped = line.strip()
    if not stripped:
        continue

    # Se a linha corresponder a um servidor, a "linha ---", a MVH ou a disciplina,
    # ela inicia um registro novo:
    if (regex.match(servidor_regex, stripped, flags=regex.IGNORECASE) or
        stripped.startswith("---") or
        regex.search(mvh_regex, stripped) or
        regex.search(disc_pattern, stripped)):
        merged_lines.append(stripped)
    else:
        # Caso contrário, concatena com a última linha adicionada
        if merged_lines:
            merged_lines[-1] = merged_lines[-1] + " " + stripped
        else:
            merged_lines.append(stripped)

lines = merged_lines

# =============================================================
#  Processamento das linhas para extração dos dados
# =============================================================
i = 0  #2.0
while i < len(lines):  #2.0
    line = lines[i].strip()  #2.0

    match_servidor = regex.match(servidor_regex, line, flags=regex.IGNORECASE)  #2.0
    if match_servidor:  #2.0
        nome_servidor = match_servidor.group(1).strip()  #2.0
        turno = match_servidor.group(2).strip()  #2.0
        ch_servidor = match_servidor.group(3).strip()  #2.0
        codigo_si = match_servidor.group(4).strip()  #2.0

        matricula = ""
        vinculo = ""
        habilitacao = ""

        if (i + 1) < len(lines):  #2.0
            line_next = lines[i + 1].strip()  #2.0
            mvh_match = regex.search(mvh_regex, line_next)  #2.0
            if mvh_match:  #2.0
                matricula = mvh_match.group(1).strip()  #2.0
                vinculo = mvh_match.group(2).strip()  #2.0
                habilitacao = mvh_match.group(3).strip()  #2.0

        j = i + 2
        while j < len(lines):  #2.0
            line_disc = lines[j].strip()  #2.0

            # Se linha de separador ou outro servidor, paramos este bloco
            if line_disc.startswith("---"):  #2.0
                break
            if regex.match(servidor_regex, line_disc, flags=regex.IGNORECASE):  #2.0
                break

            disc_matches = regex.findall(disc_pattern, line_disc)  #2.0
            if disc_matches:  #2.0
                for dm in disc_matches:  #2.0
                    ensino      = dm[0]
                    serie_ano   = dm[1]
                    turma       = dm[2]
                    cod_disc    = dm[3]
                    nome_disc   = dm[4]
                    ch_turma    = dm[5]

                    extracted_data.append({
                        "SERVIDOR": nome_servidor,
                        "TURNO": turno,
                        "CH": ch_servidor,
                        "MATRICULA": matricula,
                        "VINCULO": vinculo,
                        "HABILITACAO": habilitacao,
                        "ENSINO": ensino,
                        "SERIE/ANO": serie_ano,
                        "TURMA": turma,
                        "COD.DISC": cod_disc,
                        "DISC": nome_disc,
                        "CH.TURMA": ch_turma
                    })

            j += 1

        i = j  #2.0
    else:
        i += 1  #2.0

In [6]:
# =============================================================
# 6) CONVERTER LISTA DE DICIONÁRIOS EM DATAFRAME
# =============================================================
df = pd.DataFrame(extracted_data)

In [7]:
# =============================================================
# 7) SEPARAR "ENSINO" EM "COD.ESCOLA" e "NIVEL"
# =============================================================
df[["COD.ESCOLA", "NIVEL"]] = df["ENSINO"].str.split(n=1, expand=True)
df.drop(columns=["ENSINO"], inplace=True)

colunas_finais = [
    "SERVIDOR",
    "TURNO",
    "CH",
    "MATRICULA",
    "VINCULO",
    "HABILITACAO",
    "COD.ESCOLA",
    "NIVEL",
    "SERIE/ANO",
    "TURMA",
    "COD.DISC",
    "DISC",
    "CH.TURMA"
]
df = df[colunas_finais]

df["CH"] = pd.to_numeric(df["CH"], errors="coerce").astype("Int64")
df["CH.TURMA"] = pd.to_numeric(df["CH.TURMA"], errors="coerce").astype("Int64")

df.head(20)

saida_excel = "saida_final_ver.2.6.xlsx"
df.to_excel(saida_excel, index=False)
print(f"Planilha gerada: {saida_excel}")

Planilha gerada: saida_final_ver.2.6.xlsx


In [ ]:
# =============================================================
# EXEMPLO DE CRIAÇÃO DE OUTRA PLANILHA COM COLUNAS SEG, TER, ETC.
# =============================================================
df_filtered = df[['SERVIDOR', 'TURNO']].drop_duplicates()

dias_semana = ['SEG', 'TER', 'QUA', 'QUI', 'SEX']
for dia in dias_semana:
    for k in range(1, 6):
        df_filtered[f"{dia}{k}"] = 0

output_file = "/content/professores.xlsx"
df_filtered.to_excel(output_file, index=False)
df_filtered.head(20)